In [15]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.read_csv('hcm_houses_with_name.csv')

df['price_per_m2'] = (df['Price'] * 1000) / df['Area']

Bước 2: Chuẩn hóa dữ liệu thu được bằng z-score để loại bỏ ảnh hưởng của đơn vị đo.

In [16]:
x_bar = df['price_per_m2'].mean()
s = df['price_per_m2'].std(ddof=1)  

df['z_score'] = (df['price_per_m2'] - x_bar) / s

print(df[['Address', 'price_per_m2', 'z_score']])

                                              Address  price_per_m2   z_score
0   Đường Vườn Lài, Phường An Phú Đông, Quận 12, H...     95.238095 -0.021415
1   Đường Cư Xá Phú Lâm A, Phường 12, Quận 6, Hồ C...    137.142857  0.971376
2              Phường Thạnh Lộc, Quận 12, Hồ Chí Minh     53.797468 -1.003210
3   Dự án Melosa Garden, Đường Vành Đai Trong, Phư...     82.000000 -0.335047
4   Đường Quốc Lộ 13, Phường Hiệp Bình Phước, Thủ ...    136.446886  0.954887
..                                                ...           ...       ...
95  Đường Trần Hưng Đạo, Phường Cầu Kho, Quận 1, H...    214.285714  2.799013
96  Đường Tân Thới Hiệp 6, Phường Tân Thới Hiệp, Q...     68.421053 -0.656754
97    Đường 9, Phường Linh Xuân, Thủ Đức, Hồ Chí Minh     60.000000 -0.856262
98  Dự án Bàu Cát II, Đường Ni Sư Huỳnh Liên, Phườ...     70.290534 -0.612463
99  Đường  Phó Đức Chính, Phường 1, Bình Thạnh, Hồ...    157.407407  1.451475

[100 rows x 3 columns]


In [17]:
!pip install scipy


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Bước 3: Ước lượng khoảng tin cậy (Confidence Interval) cho giá nhà trung bình.

In [18]:
n = len(df)
x_bar = df['price_per_m2'].mean()
s = df['price_per_m2'].std(ddof=1)

confidence_level = 0.95
alpha = 1 - confidence_level
z_critical = stats.norm.ppf(1 - alpha/2)

se = s / np.sqrt(n)  
margin_error = z_critical * se

ci_lower = x_bar - margin_error
ci_upper = x_bar + margin_error

print(f"Khoảng tin cậy {confidence_level*100}% cho giá nhà trung bình:")
print(f"({ci_lower:.2f}; {ci_upper:.2f}) triệu đồng/m²")
print(f"Giá công bố của báo Tuổi Trẻ: 68 triệu đồng/m²")

Khoảng tin cậy 95.0% cho giá nhà trung bình:
(87.87; 104.41) triệu đồng/m²
Giá công bố của báo Tuổi Trẻ: 68 triệu đồng/m²


Bước 4: Thực hiện kiểm định giả thuyết thống kê để xem giá nhà trung bình hiện tại có cao hơn 68 triệu đồng/m² với mức ý nghĩa 5% hay không.

In [ ]:
n = len(df)
x_bar = df['price_per_m2'].mean()
s = df['price_per_m2'].std(ddof=1)
mu_0 = 68  

z_statistic = (x_bar - mu_0) / (s / np.sqrt(n))

p_value = 1 - stats.norm.cdf(z_statistic)

z_critical = stats.norm.ppf(0.95)  

print("=== KẾT QUẢ KIỂM ĐỊNH GIẢ THUYẾT ===")
print(f"Giá trung bình mẫu: {x_bar:.2f} triệu đồng/m²")
print(f"Giá trị giả thuyết (μ₀): {mu_0} triệu đồng/m²")
print(f"Z-score tính được: {z_statistic:.4f}")
print(f"Z-score tới hạn (α=0.05): {z_critical:.4f}")
print(f"P-value: {p_value:.10f}")
print()

if z_statistic > z_critical:
    print("KẾT LUẬN: Bác bỏ giả thuyết H₀")
    print("=> Giá nhà trung bình THỰC SỰ CAO HƠN 68 triệu đồng/m²")
else:
    print("KẾT LUẬN: Không đủ bằng chứng để bác bỏ H₀")
    print("=> Chưa thể kết luận giá nhà cao hơn 68 triệu đồng/m²")

print()
print(f"Độ chênh lệch: {x_bar - mu_0:.2f} triệu đồng/m²")
print(f"Tỷ lệ chênh lệch: {(x_bar/mu_0 - 1)*100:.1f}%")

=== KẾT QUẢ KIỂM ĐỊNH GIẢ THUYẾT ===
Giá trung bình mẫu: 96.14 triệu đồng/m²
Giá trị giả thuyết (μ₀): 68 triệu đồng/m²
Z-score tính được: 6.6673
Z-score tới hạn (α=0.05): 1.6449
P-value: 0.0000000000

KẾT LUẬN: Bác bỏ giả thuyết H₀
=> Giá nhà trung bình THỰC SỰ CAO HƠN 68 triệu đồng/m²

Độ chênh lệch: 28.14 triệu đồng/m²
Tỷ lệ chênh lệch: 41.4%
